In [24]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Создаем базу данных, в которой будем хранить информацию о песнях: айди, название, текст и ключевые слова

In [25]:
import sqlite3
con = sqlite3.connect('songs.db')
cur = con.cursor()

cur.execute("""
CREATE TABLE songs (
    song_id INT,
    name TEXT,
    lyrics TEXT,
    keywords TEXT,
    PRIMARY KEY (song_id)

)
""")
con.commit()

In [26]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
import numpy as np

#грузим стоп-слова
nltk.download('stopwords')
stops = stopwords.words()
tfidf = TfidfVectorizer(
    analyzer="word",
    stop_words=stops
)

data = []
id = 1
#для каждой песни в файле заводим переменные с названием, текстом
#и ключевыми словами
with open("lyrics.txt", "r", encoding="utf-8") as file:
    for song in file.read().split("\n\n\n"):
        name = song.split("\n")[0]
        lyrics = " ".join(song.split("\n")[1:])

        #лемматизируем текст
        morph = MorphAnalyzer()
        lyrics_preprocessed = []
        l_tokens = wordpunct_tokenize(lyrics)
        l_lemmatized = " ".join([morph.parse(item)[0].normal_form for item in l_tokens])
        lyrics_preprocessed.append(l_lemmatized)
        lyrics_tfidf = tfidf.fit_transform(lyrics_preprocessed)

        #получаем термы
        def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
            sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n + 1):-1]
            return feature_names[tfidf_vector.indices[sorted_nzs]]
        feature_names = np.array(tfidf.get_feature_names_out())
        
        #кладем в ключевые слова 20 главных термов
        words = get_top_tf_idf_words(lyrics_tfidf, feature_names, 20)
        words = str(" ".join(words))
        info = [id, name, lyrics, words]
        data.append(info)
        #увеличиваем айди для следующей песни
        id += 1

#кладем данные в базу данных
cur.executemany("INSERT or IGNORE INTO songs VALUES (?, ?, ?, ?)", data)
        
con.commit()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
